# Himanshi Sharma

## Issues Faced
I faced some issues with Neural Networks - Multilayer Perceptron Classifier, while setting baseline fully connected neural network classifier with one hidden layer.
Since we have dropped one of the variable (duration) the layers had to be changed. I just simple adjusted the layers in the function for classifer so that my code can work. To be specific I just changes 53 to 52.

# Part I
## Importing packages

In [1]:
# Importing packages necessary for building classification models
from pyspark.ml.feature import RFormula
from pyspark.ml.classification import LogisticRegression\
,LinearSVC, MultilayerPerceptronClassifier,NaiveBayes\
,GBTClassifier,RandomForestClassifier, DecisionTreeClassifier

In [2]:
## Loading the CD_samples data without duration field

In [3]:
CD_data = spark.read.csv("IS6493_lab3_CD_samples.csv",  header = True, inferSchema="true").drop("duration")
CD_data.show(5, vertical = True)

-RECORD 0---------------------
 age            | 56          
 job            | housemaid   
 marital        | married     
 education      | basic.4y    
 default        | no          
 housing        | no          
 loan           | no          
 contact        | telephone   
 month          | may         
 day_of_week    | mon         
 campaign       | 1           
 pdays          | 999         
 previous       | 0           
 poutcome       | nonexistent 
 emp_var_rate   | 1.1         
 cons_price_idx | 93.994      
 cons_conf_idx  | -36.4       
 euribor3m      | 4.857       
 nr_employed    | 5191.0      
 y              | no          
-RECORD 1---------------------
 age            | 45          
 job            | services    
 marital        | married     
 education      | basic.9y    
 default        | unknown     
 housing        | no          
 loan           | no          
 contact        | telephone   
 month          | may         
 day_of_week    | mon         
 campaig

In [4]:
# Show count by label
CD_data.groupBy('y').count().show()

+---+-----+
|  y|count|
+---+-----+
| no| 8910|
|yes| 1090|
+---+-----+



In [5]:
# Defining a RFormula 

formula = RFormula(formula="y~.",
    featuresCol="features",
    labelCol="label")

# Applying (fit) this formula to CD_data to retrieve and map CD_data's variables to the formula.
# Save the fitted formula

fitted_df = formula.fit(CD_data)

# Applying (transform) the fitted R formula to transform 
# the features and label variables in CD_data

CD_prepared = fitted_df.transform(CD_data)

In [6]:
# Viewing the first isntance in the prepared DataFrame using above created formula
CD_prepared.collect()[0]

Row(age=56, job='housemaid', marital='married', education='basic.4y', default='no', housing='no', loan='no', contact='telephone', month='may', day_of_week='mon', campaign=1, pdays=999, previous=0, poutcome='nonexistent', emp_var_rate=1.1, cons_price_idx=93.994, cons_conf_idx=-36.4, euribor3m=4.857, nr_employed=5191.0, y='no', features=SparseVector(52, {0: 56.0, 9: 1.0, 12: 1.0, 19: 1.0, 22: 1.0, 25: 1.0, 26: 1.0, 29: 1.0, 39: 1.0, 42: 1.0, 43: 999.0, 45: 1.0, 47: 1.1, 48: 93.994, 49: -36.4, 50: 4.857, 51: 5191.0}), label=0.0)

In [7]:
# meta data of features in the prepared data for classification
CD_prepared.schema["features"].metadata["ml_attr"]["attrs"]

{'binary': [{'idx': 1, 'name': 'job_admin.'},
  {'idx': 2, 'name': 'job_blue-collar'},
  {'idx': 3, 'name': 'job_technician'},
  {'idx': 4, 'name': 'job_services'},
  {'idx': 5, 'name': 'job_management'},
  {'idx': 6, 'name': 'job_retired'},
  {'idx': 7, 'name': 'job_self-employed'},
  {'idx': 8, 'name': 'job_entrepreneur'},
  {'idx': 9, 'name': 'job_housemaid'},
  {'idx': 10, 'name': 'job_unemployed'},
  {'idx': 11, 'name': 'job_student'},
  {'idx': 12, 'name': 'marital_married'},
  {'idx': 13, 'name': 'marital_single'},
  {'idx': 14, 'name': 'marital_divorced'},
  {'idx': 15, 'name': 'education_university.degree'},
  {'idx': 16, 'name': 'education_high.school'},
  {'idx': 17, 'name': 'education_basic.9y'},
  {'idx': 18, 'name': 'education_professional.course'},
  {'idx': 19, 'name': 'education_basic.4y'},
  {'idx': 20, 'name': 'education_basic.6y'},
  {'idx': 21, 'name': 'education_unknown'},
  {'idx': 22, 'name': 'default_no'},
  {'idx': 23, 'name': 'default_unknown'},
  {'idx': 24,

In [8]:
# Spliting the prepared DataFrame into train and test sets.
CD_train, CD_test = CD_prepared.randomSplit([0.7,0.3], seed = 100)

# Part 2
## (I)Building Logistic Regression Model and evaluating

In [9]:
# setting a baseline Logistic Regression classifier
lr = LogisticRegression(labelCol='label',featuresCol='features')

# Train the model using the train set
CD_LogisticRegFit = lr.fit(CD_train)

# Take a look at model's predictions for a few train instances
CD_LogisticRegFit.summary.predictions.show(5,vertical=True)
CD_LogisticRegFit.summary.predictions.select('label','prediction','probability').show(25)

#Apply the trained model on test set 
CD_LR_test_result = CD_LogisticRegFit.transform(CD_test)

# LogisticRegression has another function - evaluate to train a model 
CD_LR_evaluated = CD_LogisticRegFit.evaluate(CD_test)

# Check on the predictions for a few testing instances
CD_LR_evaluated.predictions.show(5,vertical=True)
CD_LR_evaluated.predictions.select('label','prediction','probability').show(25)


-RECORD 0------------------------------
 age            | 18                   
 job            | student              
 marital        | single               
 education      | basic.4y             
 default        | no                   
 housing        | no                   
 loan           | no                   
 contact        | cellular             
 month          | apr                  
 day_of_week    | thu                  
 campaign       | 1                    
 pdays          | 999                  
 previous       | 0                    
 poutcome       | nonexistent          
 emp_var_rate   | -1.8                 
 cons_price_idx | 93.075               
 cons_conf_idx  | -47.1                
 euribor3m      | 1.365                
 nr_employed    | 5099.1               
 y              | no                   
 features       | (52,[0,11,13,19,2... 
 label          | 0.0                  
 rawPrediction  | [1.56817077926674... 
 probability    | [0.82752268201285... 


+-----+----------+--------------------+
|label|prediction|         probability|
+-----+----------+--------------------+
|  1.0|       1.0|[0.28436741837390...|
|  1.0|       1.0|[0.25114739043467...|
|  0.0|       0.0|[0.78515595479898...|
|  0.0|       0.0|[0.81047999454064...|
|  1.0|       0.0|[0.79458275180045...|
|  0.0|       0.0|[0.56318836265077...|
|  1.0|       0.0|[0.69070611292030...|
|  0.0|       0.0|[0.51937554513234...|
|  1.0|       1.0|[0.33955851456096...|
|  1.0|       1.0|[0.30683247543202...|
|  0.0|       0.0|[0.63739551102419...|
|  0.0|       0.0|[0.69132916248523...|
|  0.0|       0.0|[0.90018309354080...|
|  0.0|       0.0|[0.80022978176089...|
|  1.0|       0.0|[0.57050757749420...|
|  0.0|       0.0|[0.57781842917337...|
|  1.0|       0.0|[0.79945437063600...|
|  0.0|       0.0|[0.95455879400744...|
|  0.0|       0.0|[0.75492296375383...|
|  0.0|       0.0|[0.97363045057638...|
|  0.0|       0.0|[0.91702477609269...|
|  1.0|       0.0|[0.61677249676387...|


In [10]:
#custom function to generate classification metrics
from pyspark.mllib.evaluation import MulticlassMetrics

def CD_classifier_evaluator (predictions):
    predictionAndLabels =predictions.select("prediction", "label").rdd
    metrics = MulticlassMetrics(predictionAndLabels)
    print("Precision(no):", round(metrics.precision(0.0),3))
    print("Recall(no):", round(metrics.recall(0.0),3))
    print("F-Score(no):", round(metrics.fMeasure(0.0, beta=1.0),3))
    print("Precision(yes):", round(metrics.precision(1.0),3))
    print("Recall(yes):", round(metrics.recall(1.0),3))
    print("F-Score(yes):", round(metrics.fMeasure(1.0, beta=1.0),3))
    print("Accuracy:", round(metrics.accuracy,3))   
    

In [11]:
CD_classifier_evaluator(CD_LR_test_result)

Precision(no): 0.913
Recall(no): 0.985
F-Score(no): 0.947
Precision(yes): 0.645
Recall(yes): 0.222
F-Score(yes): 0.33
Accuracy: 0.903


## (II) Building and evaluating a baseline decision tree model 

In [12]:
# setting a baseline decision tree classifier
dt = DecisionTreeClassifier(labelCol='label',featuresCol='features')

# Training a baseline decision tree model using the train set

CD_DT_fit = dt.fit(CD_train)

# Reviewing feature importance based on information gain
CD_DT_fit.featureImportances

SparseVector(52, {0: 0.0115, 1: 0.0054, 9: 0.004, 10: 0.0047, 16: 0.0044, 20: 0.0035, 24: 0.0041, 25: 0.0069, 26: 0.0098, 28: 0.0109, 34: 0.0259, 35: 0.0187, 37: 0.0529, 39: 0.008, 42: 0.0067, 43: 0.1623, 46: 0.0122, 48: 0.0117, 49: 0.0056, 50: 0.0325, 51: 0.5984})

In [13]:
# Applying the baseline model and generate classification metrics
CD_DT_test_results = CD_DT_fit.transform(CD_test)

# generating classification metrics

CD_classifier_evaluator(CD_DT_test_results)

Precision(no): 0.911
Recall(no): 0.985
F-Score(no): 0.947
Precision(yes): 0.626
Recall(yes): 0.209
F-Score(yes): 0.314
Accuracy: 0.901


## (III) Building and evaluating a baseline Random Forest model

In [14]:
# setting a baseline Random Forest classifier
rfc = RandomForestClassifier(labelCol='label',featuresCol='features')

# Training a baseline Random Forest model using the train set
CD_RFC_fit = rfc.fit(CD_train)

# Reviewing feature importance based on information gain
CD_RFC_fit.featureImportances

SparseVector(52, {0: 0.0322, 1: 0.0044, 2: 0.0019, 3: 0.001, 4: 0.0019, 5: 0.0017, 6: 0.004, 7: 0.0009, 8: 0.0007, 9: 0.0001, 10: 0.0008, 11: 0.0006, 12: 0.0027, 13: 0.0032, 14: 0.0008, 15: 0.0031, 16: 0.0017, 17: 0.0015, 18: 0.0015, 19: 0.0011, 20: 0.0009, 21: 0.0004, 22: 0.003, 23: 0.0071, 24: 0.0015, 25: 0.0002, 26: 0.001, 27: 0.0026, 28: 0.022, 29: 0.002, 30: 0.0003, 31: 0.0013, 32: 0.0045, 33: 0.0011, 34: 0.0045, 35: 0.0128, 36: 0.0064, 37: 0.0391, 38: 0.0052, 39: 0.0013, 40: 0.0015, 41: 0.003, 42: 0.0097, 43: 0.2244, 44: 0.0279, 45: 0.0242, 46: 0.0302, 47: 0.0473, 48: 0.0612, 49: 0.0277, 50: 0.1839, 51: 0.1757})

In [15]:
# Applying the baseline model to the test set and generate classification metrics

CD_RFC_test_results = CD_RFC_fit.transform(CD_test)

# generating classification metrics

CD_classifier_evaluator(CD_RFC_test_results)

Precision(no): 0.907
Recall(no): 0.99
F-Score(no): 0.946
Precision(yes): 0.654
Recall(yes): 0.159
F-Score(yes): 0.256
Accuracy: 0.9


## (IV) Building and evaluating a baseline Gradient Boosting Tree model

In [16]:
# setting a baseline Gradient Boosting Tree (GBT) classifier
gbt = GBTClassifier(labelCol='label',featuresCol='features')

# Training a baseline GBT model using the train set
CD_GBT_fit = gbt.fit(CD_train)

# We can review the importance of features, typically determined based on information gain
CD_GBT_fit.featureImportances

SparseVector(52, {0: 0.1412, 1: 0.0179, 2: 0.0182, 3: 0.0098, 4: 0.0157, 5: 0.0136, 6: 0.0047, 7: 0.0121, 8: 0.001, 9: 0.0027, 10: 0.0254, 11: 0.0, 12: 0.0132, 13: 0.0106, 14: 0.0046, 15: 0.0076, 16: 0.0048, 17: 0.0094, 18: 0.0006, 19: 0.0042, 20: 0.0002, 21: 0.0081, 22: 0.0137, 24: 0.0178, 25: 0.0085, 26: 0.0157, 27: 0.0191, 28: 0.0223, 30: 0.0051, 31: 0.0072, 32: 0.0063, 33: 0.0102, 34: 0.0101, 35: 0.0141, 36: 0.0053, 37: 0.01, 38: 0.0141, 39: 0.0037, 40: 0.0127, 41: 0.0066, 42: 0.062, 43: 0.0567, 44: 0.0239, 45: 0.0045, 46: 0.0247, 47: 0.0546, 48: 0.0423, 49: 0.0283, 50: 0.1316, 51: 0.0437})

In [17]:
# Applying the baseline model to the test set and generating classification metrics

CD_GBT_test_results = CD_GBT_fit.transform(CD_test)
CD_classifier_evaluator(CD_GBT_test_results)

Precision(no): 0.918
Recall(no): 0.977
F-Score(no): 0.946
Precision(yes): 0.593
Recall(yes): 0.278
F-Score(yes): 0.379
Accuracy: 0.901


## (V)Building and evaluating a baseline Support Vector Machine model

In [18]:
# setting a baseline LinearSVC classifier
svm = LinearSVC(labelCol='label',featuresCol='features')

# Training a baseline SVM model using the train set
CD_SVM_Fit = svm.fit(CD_train)

In [19]:
# Applying the baseline model to the test set and generating classification metrics
CD_SVM_test_results = CD_SVM_Fit.transform(CD_test)
CD_classifier_evaluator(CD_SVM_test_results)

Precision(no): 0.895
Recall(no): 0.993
F-Score(no): 0.941
Precision(yes): 0.406
Recall(yes): 0.041
F-Score(yes): 0.074
Accuracy: 0.89


## (VI)Building and evaluating a baseline Naive Bayes model

In [20]:
# Define an RFormula for a Naive Bayes model by removing 'emp_var_rate' and 'cons_conf_idx' that contain negative values.

formulaNB = RFormula(formula="y~.-emp_var_rate-cons_conf_idx",
    featuresCol="features",
    labelCol="label")

CD_NB_fit = formulaNB.fit(CD_data)

# Transform CD_data to be in format compatible with pyspark
CD_NB_df_prepared = CD_NB_fit.transform(CD_data)

In [21]:
CD_NB_df_prepared.collect()[0]
# View the meta data of features in the prepared data for classification
CD_NB_df_prepared.schema["features"].metadata["ml_attr"]["attrs"]

{'binary': [{'idx': 1, 'name': 'job_admin.'},
  {'idx': 2, 'name': 'job_blue-collar'},
  {'idx': 3, 'name': 'job_technician'},
  {'idx': 4, 'name': 'job_services'},
  {'idx': 5, 'name': 'job_management'},
  {'idx': 6, 'name': 'job_retired'},
  {'idx': 7, 'name': 'job_self-employed'},
  {'idx': 8, 'name': 'job_entrepreneur'},
  {'idx': 9, 'name': 'job_housemaid'},
  {'idx': 10, 'name': 'job_unemployed'},
  {'idx': 11, 'name': 'job_student'},
  {'idx': 12, 'name': 'marital_married'},
  {'idx': 13, 'name': 'marital_single'},
  {'idx': 14, 'name': 'marital_divorced'},
  {'idx': 15, 'name': 'education_university.degree'},
  {'idx': 16, 'name': 'education_high.school'},
  {'idx': 17, 'name': 'education_basic.9y'},
  {'idx': 18, 'name': 'education_professional.course'},
  {'idx': 19, 'name': 'education_basic.4y'},
  {'idx': 20, 'name': 'education_basic.6y'},
  {'idx': 21, 'name': 'education_unknown'},
  {'idx': 22, 'name': 'default_no'},
  {'idx': 23, 'name': 'default_unknown'},
  {'idx': 24,

In [22]:
# Split the reformatted data into train and test sets
CD_NB_train, CD_NB_test = CD_NB_df_prepared.randomSplit([0.7,0.3], seed = 100)

# set an nb classifier
nb = NaiveBayes(labelCol='label',featuresCol='features')
# Train an nb model
CD_NB_fit = nb.fit(CD_NB_train)

In [23]:
# Apply the baseline model to the test set and generate classification metrics
CD_NB_test_results = CD_NB_fit.transform(CD_NB_test)
CD_classifier_evaluator(CD_NB_test_results)

Precision(no): 0.91
Recall(no): 0.984
F-Score(no): 0.946
Precision(yes): 0.6
Recall(yes): 0.197
F-Score(yes): 0.296
Accuracy: 0.899


## (VII) Building and Evaluating a simple Neural Network model

In [24]:

# setting a baseline fully connected neural network classifier with one hidden layer
mlp = MultilayerPerceptronClassifier(labelCol='label',featuresCol='features', layers =[52,27,2])

# Train the model using the train set
CD_MLP_fit = mlp.fit(CD_train)

# Apply the trained model to the test set to generate predictions
CD_MLP_test_results = CD_MLP_fit.transform(CD_test)

# Use the customized function to generate testing metrics of the trained mlp model
CD_classifier_evaluator(CD_MLP_test_results)

Precision(no): 0.892
Recall(no): 1.0
F-Score(no): 0.943
Precision(yes): 0.0
Recall(yes): 0.0
F-Score(yes): 0.0
Accuracy: 0.892


In [25]:
# Defining a function classifierEvaluatorLine

def CD_classifier_evaluator_line (model,predictions):
    predictionAndLabels =predictions.select("prediction", "label").rdd
    metrics = MulticlassMetrics(predictionAndLabels)
    print(model,"PRC-n:", round(metrics.precision(0.0),3)\
          ,"TPR-n:", round(metrics.recall(0.0),3)\
          ,"F1-n:", round(metrics.fMeasure(0.0, beta=1.0),3)\
          ,"PRC-y:", round(metrics.precision(1.0),3)\
          ,"TPR-y:", round(metrics.recall(1.0),3)\
          ,"F1-y:", round(metrics.fMeasure(1.0, beta=1.0),3)\
          ,"ACC:", round(metrics.accuracy,3))

In [26]:
# Call CD_classifierEvaluatorLine with each of the saved testing results as input
CD_classifier_evaluator_line("LR     ", CD_LR_test_result)

CD_classifier_evaluator_line("DT     ", CD_DT_test_results)

CD_classifier_evaluator_line("RF     ", CD_RFC_test_results)

CD_classifier_evaluator_line("GBT   ", CD_GBT_test_results)

CD_classifier_evaluator_line("SVM    ", CD_SVM_test_results)

CD_classifier_evaluator_line("NB-rfNB", CD_NB_test_results)

CD_classifier_evaluator_line("MLP    ", CD_MLP_test_results)


LR      PRC-n: 0.913 TPR-n: 0.985 F1-n: 0.947 PRC-y: 0.645 TPR-y: 0.222 F1-y: 0.33 ACC: 0.903
DT      PRC-n: 0.911 TPR-n: 0.985 F1-n: 0.947 PRC-y: 0.626 TPR-y: 0.209 F1-y: 0.314 ACC: 0.901
RF      PRC-n: 0.907 TPR-n: 0.99 F1-n: 0.946 PRC-y: 0.654 TPR-y: 0.159 F1-y: 0.256 ACC: 0.9
GBT    PRC-n: 0.918 TPR-n: 0.977 F1-n: 0.946 PRC-y: 0.593 TPR-y: 0.278 F1-y: 0.379 ACC: 0.901
SVM     PRC-n: 0.895 TPR-n: 0.993 F1-n: 0.941 PRC-y: 0.406 TPR-y: 0.041 F1-y: 0.074 ACC: 0.89
NB-rfNB PRC-n: 0.91 TPR-n: 0.984 F1-n: 0.946 PRC-y: 0.6 TPR-y: 0.197 F1-y: 0.296 ACC: 0.899
MLP     PRC-n: 0.892 TPR-n: 1.0 F1-n: 0.943 PRC-y: 0.0 TPR-y: 0.0 F1-y: 0.0 ACC: 0.892


# Part 3: Multiclass Classification Model Building and Evaluating

In [27]:
# importing iris dataset
iris_df = spark.read.csv("iris_data.csv", header = True, inferSchema="true")

In [28]:
# examine count of instances by class
iris_df.groupBy('plant').count().show()

+---------------+-----+
|          plant|count|
+---------------+-----+
| Iris-virginica|   50|
|    Iris-setosa|   50|
|Iris-versicolor|   50|
+---------------+-----+



## Prepare classification data using RFormula

In [29]:
# Defining an RFormula 

formula_iris = RFormula(formula="plant~.",
    featuresCol="features",
    labelCol="label")

# Applying (using fit) this formula to iris_df to retrieve and map iris_df's variables to the formula.
# Saving the fitted formula

fitted_rf_iris = formula_iris.fit(iris_df)

# Applying (transform) the fitted R formula to transform 
# the features and label variables in iris_df

iris_prepared = fitted_rf_iris.transform(iris_df)

In [30]:
# View the first record the prepared data
iris_prepared.collect()[0]

Row(sepLength=5.1, sepWidth=3.5, petLength=1.4, petWidth=0.2, plant='Iris-setosa', features=DenseVector([5.1, 3.5, 1.4, 0.2]), label=0.0)

In [31]:
# View the meta data of features in the prepared data for classification
iris_prepared.schema["features"].metadata["ml_attr"]["attrs"]

{'numeric': [{'idx': 0, 'name': 'sepLength'},
  {'idx': 1, 'name': 'sepWidth'},
  {'idx': 2, 'name': 'petLength'},
  {'idx': 3, 'name': 'petWidth'}]}

In [32]:
# Split the prepared data into train and test DataFrames
iris_train, iris_test = iris_prepared.randomSplit([0.7,0.3], seed=100)

In [33]:
from pyspark.mllib.evaluation import MulticlassMetrics

def iris_classifier_evaluator (predictions):
    predictionAndLabels =predictions.select("prediction", "label").rdd
    metrics = MulticlassMetrics(predictionAndLabels)
    print("Precision(class1):", round(metrics.precision(0.0),3))
    print("Recall(class1):", round(metrics.recall(0.0),3))
    print("F-Score(class1):", round(metrics.fMeasure(0.0, beta=1.0),3))
    print("Precision(class2):", round(metrics.precision(1.0),3))
    print("Recall(class2):", round(metrics.recall(1.0),3))
    print("F-Score(class2):", round(metrics.fMeasure(1.0, beta=1.0),3))
    print("Precision(class3):", round(metrics.precision(2.0),3))
    print("Recall(class3):", round(metrics.recall(2.0),3))
    print("F-Score(class3):", round(metrics.fMeasure(2.0, beta=1.0),3))
    print("Accuracy:", round(metrics.accuracy,3))   
    

In [34]:
# Applying the baseline Logistic method 
lr = LogisticRegression(labelCol='label',featuresCol='features')
iris_LR_fit = lr.fit(iris_train)

# Applying the trained model to the test set to generate predictions
iris_LR_test_results = iris_LR_fit.transform(iris_test)

# Use the customized function to generate testing metrics of the trained model
iris_classifier_evaluator(iris_LR_test_results)

Precision(class1): 1.0
Recall(class1): 0.947
F-Score(class1): 0.973
Precision(class2): 0.929
Recall(class2): 1.0
F-Score(class2): 0.963
Precision(class3): 1.0
Recall(class3): 1.0
F-Score(class3): 1.0
Accuracy: 0.979


In [35]:
## setting a baseline fully connected neural network classifier with one hidden layer. 
mlp = MultilayerPerceptronClassifier(labelCol='label',featuresCol='features',layers=[4,2,3])

# Train the model using the train set
iris_MLP_fit = mlp.fit(iris_train)

# Apply the trained model to the test set to generate predictions
iris_MLP_test_results = iris_MLP_fit.transform(iris_test)

# Use the customized function to generate testing metrics of the trained mlp model
iris_classifier_evaluator(iris_MLP_test_results)

Precision(class1): 1.0
Recall(class1): 1.0
F-Score(class1): 1.0
Precision(class2): 1.0
Recall(class2): 0.923
F-Score(class2): 0.96
Precision(class3): 0.938
Recall(class3): 1.0
F-Score(class3): 0.968
Accuracy: 0.979


In [36]:
# set a baseline decision tree classifier
dt = DecisionTreeClassifier(labelCol='label',featuresCol='features')

# Train a baseline decision tree model using the train set
iris_DT_fit = dt.fit(iris_train)

# We can review the importance of features, typically determined based on information gain
iris_DT_fit.featureImportances

# Apply the baseline model and generate classification metrics

iris_DT_test_results = iris_DT_fit.transform(iris_test)

# Call the CD_classifier_evaluator function to generate classification metrics

iris_classifier_evaluator(iris_DT_test_results)

Precision(class1): 1.0
Recall(class1): 1.0
F-Score(class1): 1.0
Precision(class2): 0.929
Recall(class2): 1.0
F-Score(class2): 0.963
Precision(class3): 1.0
Recall(class3): 0.933
F-Score(class3): 0.966
Accuracy: 0.979


In [37]:
# setting a baseline Random Forest classifier
rfc = RandomForestClassifier(labelCol='label',featuresCol='features')

# Training a baseline Random Forest model using the train set

iris_RFC_fit = rfc.fit(iris_train)

# We can review the importance of features, typically determined based on information gain
iris_RFC_fit.featureImportances

SparseVector(4, {0: 0.1317, 1: 0.0466, 2: 0.3698, 3: 0.4519})

In [38]:
# Applying the baseline model to the test set
iris_RFC_test_results = iris_RFC_fit.transform(iris_test)

# generating classification metrics

iris_classifier_evaluator(iris_RFC_test_results)

Precision(class1): 1.0
Recall(class1): 1.0
F-Score(class1): 1.0
Precision(class2): 0.867
Recall(class2): 1.0
F-Score(class2): 0.929
Precision(class3): 1.0
Recall(class3): 0.867
F-Score(class3): 0.929
Accuracy: 0.957


In [39]:
# None of the variable in iris data set are negative so using previously defined
# iris_train and iris_test

# setting an nb classifier
nb = NaiveBayes(labelCol='label',featuresCol='features')
# Training an nb model
iris_NB_fit = nb.fit(iris_train)

# Applying the baseline model to the test set and generate classification metrics
iris_NB_test_results = iris_NB_fit.transform(iris_test)
iris_classifier_evaluator(iris_NB_test_results)

Precision(class1): 1.0
Recall(class1): 1.0
F-Score(class1): 1.0
Precision(class2): 0.812
Recall(class2): 1.0
F-Score(class2): 0.897
Precision(class3): 1.0
Recall(class3): 0.8
F-Score(class3): 0.889
Accuracy: 0.936


In [40]:
# Define a function classifierEvaluatorLine

def iris_classifier_evaluator_line (model,predictions):
    predictionAndLabels =predictions.select("prediction", "label").rdd
    metrics = MulticlassMetrics(predictionAndLabels)
    print(model,"PRC-n:", round(metrics.precision(0.0),3)\
          ,"TPR-n:", round(metrics.recall(0.0),3)\
          ,"F1-n:", round(metrics.fMeasure(0.0, beta=1.0),3)\
          ,"PRC-y:", round(metrics.precision(1.0),3)\
          ,"TPR-y:", round(metrics.recall(1.0),3)\
          ,"F1-y:", round(metrics.fMeasure(1.0, beta=1.0),3)\
          ,"ACC:", round(metrics.accuracy,3))

In [41]:
iris_classifier_evaluator_line("LR     ", iris_LR_test_results)

iris_classifier_evaluator_line("DT     ", iris_DT_test_results)

iris_classifier_evaluator_line("RF     ", iris_RFC_test_results)


iris_classifier_evaluator_line("NB-rfNB", iris_NB_test_results)

iris_classifier_evaluator_line("MLP    ", iris_MLP_test_results)

LR      PRC-n: 1.0 TPR-n: 0.947 F1-n: 0.973 PRC-y: 0.929 TPR-y: 1.0 F1-y: 0.963 ACC: 0.979
DT      PRC-n: 1.0 TPR-n: 1.0 F1-n: 1.0 PRC-y: 0.929 TPR-y: 1.0 F1-y: 0.963 ACC: 0.979
RF      PRC-n: 1.0 TPR-n: 1.0 F1-n: 1.0 PRC-y: 0.867 TPR-y: 1.0 F1-y: 0.929 ACC: 0.957
NB-rfNB PRC-n: 1.0 TPR-n: 1.0 F1-n: 1.0 PRC-y: 0.812 TPR-y: 1.0 F1-y: 0.897 ACC: 0.936
MLP     PRC-n: 1.0 TPR-n: 1.0 F1-n: 1.0 PRC-y: 1.0 TPR-y: 0.923 F1-y: 0.96 ACC: 0.979
